In [9]:
import os
import time
from datetime import datetime
import pandas as pd

import cv2
import yt_dlp
from ultralytics import YOLO
from collections import defaultdict
import supervision as sv


In [10]:
video_url = "https://www.youtube.com/watch?v=0Pg3S6s76IE"  # Youtube URL northbound


In [11]:
def get_stream_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'skip_download': True,
        'no_warnings': True,
        'force_generic_extractor': False,
        'format': 'best[ext=mp4][protocol^=http]/best'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        formats = info.get('formats', [info])
        # prefer video-only best mp4 OR best overall
        for f in reversed(formats):
            if f.get('vcodec', '') != 'none' and f.get('acodec', '') == 'none' and 'url' in f:
                return f['url']
        # fallback: best available format
        return formats[-1]['url']

In [ ]:
# Load the YOLO model
model = YOLO('yolo11l.pt')

class_list = model.names 

# Dictionary to store object counts by class
class_counts = defaultdict(int)

# the line
line_y_red_end = 590  # Red line position
line_y_red_start = 960  # Red line position


def capture_frames(stream_url):

    # Open the video file
    cap = cv2.VideoCapture(stream_url)

    frame_count = 0
    resultsList = []

    while True:
        
        ret, frame = cap.read() # read each frame one by one
        
        if not ret: #when there are no more frames, break
            # Release resources on way out of while loop
            print("on way to first break")
            cap.release()
            cv2.destroyAllWindows()
            break

        # method to set frame position with cap.set and skip frames wasn't really working
        # going to try this method to process only every tenth frame... let's try every 15th now
        frame_count += 1
        if frame_count % 15 != 0:
            continue  # Skip frames that are not multiples of 15
        
        # Run YOLO tracking on the frame
        results = model.track(frame, classes = [1,2,3,5,6,7], persist=True) 
        #print(results)

        # Ensure results are not empty
        if results[0].boxes.data is not None:
            # Get the detected boxes, their class indices, and track IDs
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            class_indices = results[0].boxes.cls.int().cpu().tolist()
            confidences = results[0].boxes.conf.cpu()

            cv2.line(frame, (400, line_y_red_end), (1450, line_y_red_end), (0, 0, 255), 1)
            cv2.line(frame, (400, line_y_red_start), (1450, line_y_red_start), (0, 0, 255), 1)
            #cv2.putText(frame, 'Red Line', (690, line_y_red - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
            # Loop through each detected object
            for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
                x1, y1, x2, y2 = map(int, box)
                cx = (x1 + x2) // 2  # Calculate the center point
                cy = (y1 + y2) // 2            

                class_name = class_list[class_idx]

                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                
                cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 

                dfKeyFeatures = pd.DataFrame({'id' : [track_id], 
                                              'class' : [class_name], 
                                              'confidence' : [conf], 
                                              'cx' : [cx], 
                                              'cy' : [cy], 
                                              'timestamp' : [cap.get(cv2.CAP_PROP_POS_MSEC)]})
                resultsList.append(dfKeyFeatures)

            # Display the counts on the frame
            y_offset = 50
            for class_name, count in class_counts.items():
                cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                y_offset += 30

        ### Save frame    
        # prep timestamp for file name
        current_datetime = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
        # Save as PNG
        filename =r'c:\Users\Nineveh.OConnell\OneDrive - DOT OST\Documents\code\Border-Bottleneck-Management\support_scripts\captured_pngs'
        cv2.imwrite(f"{filename}/frame_{str(current_datetime)}.png", frame)
        print(f"Saved frame {current_datetime}")

        #i += 1
        get_curr_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)
        print(get_curr_pos)
        #cap.set(cv2.CAP_PROP_POS_FRAMES, get_curr_pos + 15) # skip frames, 1 corresponds to the propid index, 10 skips 10 frames I believe
        print(cap.get(cv2.CAP_PROP_POS_FRAMES))

        # Show the frame
        cv2.imshow("YOLO Object Tracking & Counting", frame)    

        # Exit loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'): #removed:  
            # Release resources on way out of while loop
            print("on way to second break")
            cap.release()
            cv2.destroyAllWindows()
            break

    # Save as data frame
    combined_df = pd.concat(resultsList)
    datafolder =r'c:\Users\Nineveh.OConnell\OneDrive - DOT OST\Documents\code\Border-Bottleneck-Management\support_scripts\captured_coords'
    combined_df.to_csv(f"{datafolder}/data_download_export.csv", index=False) 


if __name__ == "__main__":
    stream_url = get_stream_url(video_url)
    print("Stream URL:", stream_url)
    capture_frames(stream_url)

    


Stream URL: https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1752281197/ei/DVxxaNiSMpqP2_gPqqbx8Q4/ip/152.122.255.5/id/0Pg3S6s76IE.2/itag/270/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hls_chunk_host/rr1---sn-vgqsknsk.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/bui/AY1jyLPXfnyJog0qUVHlWIIpo3-QnyCJbicaQCHr-j64LBaen9hdNJC91o1Zd52W_LFs02pXubTydLag/spc/l3OVKbnhG6SQHZehihX_cymWCE7Jm65Fht6z0x47jEJcWuwsi785mapNtoE/vprv/1/playlist_type/DVR/met/1752259598,/mh/sg/mm/44/mn/sn-vgqsknsk/ms/lva/mv/u/mvi/1/pl/16/rms/lva,lva/dover/13/pacing/0/short_key/1/keepalive/yes/mt/1752257996/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgovp,rqh,xpc,playlist_duration,manifest_duration,bui,spc,vprv,playlist_type/sig/AJfQdSswRgIhALMktlurwzpVO0mkjOC2bBEncdixl5GmMU7LvJQxofPgAiEArrYkendVhHfzfkqX22QDtHur5rV7lLdHpnVdj3tEsQ4%3D/lsparams/hls_chunk_host,met,mh,mm,mn,ms,mv,mvi,pl,rms/lsig/APaTxxMwRQI